## <a name="C4"> Projet 4-3 : Variable Cible SiteEnergyUse(kBtu)</a>

## <a name="C4"> Mise en Place de l'Environnement de Travail</a>

Nous commençons par configurer notre environnement de travail en important les librairies nécessaires.

- Installation des Librairies

In [ ]:
# Importation des librairies nécessaires pour le projet

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns  
from scipy import stats  
import missingno as msno
import os
os.environ['OMP_NUM_THREADS'] = '1'
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import normaltest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.stats import boxcox
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import t
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score

# Afficher les graphiques directement dans le notebook
%matplotlib inline

# Configuration de Seaborn pour améliorer l'esthétique des graphiques
sns.set(style="whitegrid")

# Afficher plus de lignes et de colonnes
pd.set_option('display.max_row',1000)
pd.set_option('display.max_column',300)

# Indication d'importations réussies
print("Librairies importées avec succès !")

In [ ]:
print("Version des librairies utilisées")
print("--------------------------------")
print("Version de Pandas:", pd.__version__)
print("Version de NumPy:", np.__version__)
print("Version de Matplotlib:", plt.matplotlib.__version__)
print("Version de Seaborn:", sns.__version__)
print("Version de Missingno:", msno.__version__)
print("Version de Statsmodels:", sm.__version__)

- Lecture des fichiers de données

Nous lisons ensuite les fichiers de données issus du site de seattle.gov.

In [ ]:
# Chargement des fichiers CSV dans des DataFrames pandas
data = pd.read_csv('data.csv', delimiter=';')
data_bis = pd.read_csv('data.csv', delimiter=';')
# Indication d'importations réussies
print("Fichier importé avec succès !")

## <a name="C4"> Modélisation </a>


In [ ]:
# Remplacer les valeurs manquantes dans SecondLargestPropertyUseType par "None"
data['SecondLargestPropertyUseType'].fillna("None", inplace=True)

# Remplacer les valeurs manquantes dans SecondLargestPropertyUseTypeGFA par 0
data['SecondLargestPropertyUseTypeGFA'].fillna(0, inplace=True)

# Remplacer les valeurs manquantes dans ThirdLargestPropertyUseType par "None"
data['ThirdLargestPropertyUseType'].fillna("None", inplace=True)

# Remplacer les valeurs manquantes dans ThirdLargestPropertyUseTypeGFA par 0
data['ThirdLargestPropertyUseTypeGFA'].fillna(0, inplace=True)

#### <font color='skyblue'> Préparation des données d'entrainement et de test</font>

In [ ]:
data.shape

In [ ]:
data.head(3)

In [ ]:
columns_to_drop = ['OSEBuildingID', 'PropertyName', 'Address','Unnamed: 0','CouncilDistrictCode',
                   'TaxParcelIdentificationNumber','YearBuilt','ListOfAllPropertyUseTypes',
                   'SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType', 'PrimaryPropertyType',
                   'CouncilDistrictCode','YearBuilt','PropertyGFABuilding(s)',
                   'LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseTypeGFA',
                    'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)','SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 
                    'Electricity(kWh)', 'NaturalGas(therms)',
                    'GHGEmissionsIntensity','SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)','LargestPropertyUseType',
                    'BuildingType','PropertyGFAParking']
data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
data

In [ ]:
categorical_data = ['Category', 'Neighborhood']
numerical_data = ['NumberofFloors','NumberofBuildings','PropertyGFATotal','BuildingAge','GasUseRatio','ElectricityUseRatio','ParkingGFARatio']

#### <font color='skyblue'> Division des données et entrainement </font>


In [ ]:
X = data.drop(['TotalGHGEmissions','SiteEnergyUse(kBtu)'], axis=1)

y = data[['TotalGHGEmissions','SiteEnergyUse(kBtu)']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

print('X_train :', X_train.shape)
print('y_train :', y_train.shape)
print('X_test :' , X_test.shape)
print('y_test :',y_test.shape)

#### <font color='skyblue'> Régression linéaire </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de regression linéaire
linear_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('linear_regression', LinearRegression())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
linear_param_grid = {}
linear_grid_search = GridSearchCV(linear_pipeline, 
                                  param_grid=linear_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
linear_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = linear_grid_search.best_score_
best_mae_score = -linear_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][linear_grid_search.best_index_]
best_rmse_score = -linear_grid_search.cv_results_['mean_test_neg_mean_squared_error'][linear_grid_search.best_index_]

# Affichage des résultats
print("Régression linéaire")
print("-------------------")
print("- Meilleurs hyperparamètres:", linear_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", linear_grid_search.refit_time_)


In [ ]:
cv_result=pd.DataFrame(linear_grid_search.cv_results_)
cv_result


- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de regression linéaire
linear_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('linear_regression', LinearRegression())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
linear_param_grid = {}
linear_grid_search = GridSearchCV(linear_pipeline, 
                                  param_grid=linear_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
linear_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = linear_grid_search.best_score_
best_mae_score = -linear_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][linear_grid_search.best_index_]
best_rmse_score = -linear_grid_search.cv_results_['mean_test_neg_mean_squared_error'][linear_grid_search.best_index_]

# Affichage des résultats
print("Régression linéaire logarithmique")
print("-------------------")
print("- Meilleurs hyperparamètres:", linear_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", linear_grid_search.refit_time_)


#### <font color='skyblue'> Dummy Regressor </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression dummy
dummy_pipeline = Pipeline([('preprocessor', preprocessor),
                           ('dummy_regressor', DummyRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
dummy_param_grid = {'dummy_regressor__strategy': ['mean', 'median', 'quantile'],
                    'dummy_regressor__quantile': [0.25, 0.5, 0.75]}
dummy_grid_search = GridSearchCV(dummy_pipeline, 
                                  param_grid=dummy_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
dummy_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = dummy_grid_search.best_score_
best_mae_score = -dummy_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][dummy_grid_search.best_index_]
best_rmse_score = -dummy_grid_search.cv_results_['mean_test_neg_mean_squared_error'][dummy_grid_search.best_index_]

# Affichage des résultats
print("Dummy Regressor")
print("-------------------")
print("- Meilleurs hyperparamètres:", dummy_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", dummy_grid_search.refit_time_)

- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression dummy
dummy_pipeline = Pipeline([('preprocessor', preprocessor),
                           ('dummy_regressor', DummyRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
dummy_param_grid = {'dummy_regressor__strategy': ['mean', 'median', 'quantile'],
                    'dummy_regressor__quantile': [0.25, 0.5, 0.75]}
dummy_grid_search = GridSearchCV(dummy_pipeline, 
                                  param_grid=dummy_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
dummy_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = dummy_grid_search.best_score_
best_mae_score = -dummy_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][dummy_grid_search.best_index_]
best_rmse_score = -dummy_grid_search.cv_results_['mean_test_neg_mean_squared_error'][dummy_grid_search.best_index_]

# Affichage des résultats
print("Dummy Regressor")
print("-------------------")
print("- Meilleurs hyperparamètres:", dummy_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", dummy_grid_search.refit_time_)

#### <font color='skyblue'> Régression Lasso </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression lasso
lasso_pipeline  = Pipeline([('preprocessor', preprocessor),
                           ('lasso_regression', Lasso())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
lasso_param_grid = {"lasso_regression__alpha": [0.1, 1.0, 10.0],
                    "lasso_regression__fit_intercept": [True, False]}
lasso_grid_search = GridSearchCV(lasso_pipeline , 
                                  param_grid=lasso_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
lasso_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = lasso_grid_search.best_score_
best_mae_score = -lasso_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][lasso_grid_search.best_index_]
best_rmse_score = -lasso_grid_search.cv_results_['mean_test_neg_mean_squared_error'][lasso_grid_search.best_index_]

# Affichage des résultats
print("Régression Lasso")
print("-------------------")
print("- Meilleurs hyperparamètres:", lasso_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", lasso_grid_search.refit_time_)

- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression lasso
lasso_pipeline  = Pipeline([('preprocessor', preprocessor),
                           ('lasso_regression', Lasso())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
lasso_param_grid = {"lasso_regression__alpha": [0.1, 1.0, 10.0],
                    "lasso_regression__fit_intercept": [True, False]}
lasso_grid_search = GridSearchCV(lasso_pipeline , 
                                  param_grid=lasso_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
lasso_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = lasso_grid_search.best_score_
best_mae_score = -lasso_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][lasso_grid_search.best_index_]
best_rmse_score = -lasso_grid_search.cv_results_['mean_test_neg_mean_squared_error'][lasso_grid_search.best_index_]

# Affichage des résultats
print("Régression Lasso")
print("-------------------")
print("- Meilleurs hyperparamètres:", lasso_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", lasso_grid_search.refit_time_)

#### <font color='skyblue'> Régression Ridge </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression Ridge
ridge_pipeline = Pipeline([('preprocessor', preprocessor),
                           ('ridge_regression', Ridge())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
ridge_param_grid = {"ridge_regression__alpha": [0.1, 1.0, 10.0],
                    "ridge_regression__fit_intercept": [True, False]}
ridge_grid_search = GridSearchCV(ridge_pipeline , 
                                  param_grid=ridge_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
ridge_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = ridge_grid_search.best_score_
best_mae_score = -ridge_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ridge_grid_search.best_index_]
best_rmse_score = -ridge_grid_search.cv_results_['mean_test_neg_mean_squared_error'][ridge_grid_search.best_index_]

# Affichage des résultats
print("Régression Ridge")
print("-------------------")
print("- Meilleurs hyperparamètres:", ridge_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", ridge_grid_search.refit_time_)

- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de régression Ridge
ridge_pipeline = Pipeline([('preprocessor', preprocessor),
                           ('ridge_regression', Ridge())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
ridge_param_grid = {"ridge_regression__alpha": [0.1, 1.0, 10.0],
                    "ridge_regression__fit_intercept": [True, False]}
ridge_grid_search = GridSearchCV(ridge_pipeline , 
                                  param_grid=ridge_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
ridge_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = ridge_grid_search.best_score_
best_mae_score = -ridge_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ridge_grid_search.best_index_]
best_rmse_score = -ridge_grid_search.cv_results_['mean_test_neg_mean_squared_error'][ridge_grid_search.best_index_]

# Affichage des résultats
print("Régression Ridge")
print("-------------------")
print("- Meilleurs hyperparamètres:", ridge_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", ridge_grid_search.refit_time_)

#### <font color='skyblue'> Random Forest </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Random Forest
forest_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('random_forest', RandomForestRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
forest_param_grid = {'random_forest__max_features' : [0.8],
                    'random_forest__max_depth': [5, 10, 15],
                    'random_forest__min_samples_split': [2, 5, 10],
                    'random_forest__bootstrap' : [True, False],
                    'random_forest__min_samples_leaf': [1,2,5,10]}
forest_grid_search = GridSearchCV(forest_pipeline, 
                                  param_grid=forest_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
forest_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = forest_grid_search.best_score_
best_mae_score = -forest_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][forest_grid_search.best_index_]
best_rmse_score = -forest_grid_search.cv_results_['mean_test_neg_mean_squared_error'][forest_grid_search.best_index_]

# Affichage des résultats
print("Random Forest")
print("-------------------")
print("- Meilleurs hyperparamètres:", forest_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", forest_grid_search.refit_time_  )


- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Random Forest
forest_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('random_forest', RandomForestRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
forest_param_grid = {'random_forest__max_features' : [0.8],
                    'random_forest__max_depth': [5, 10, 15],
                    'random_forest__min_samples_split': [2, 5, 10],
                    'random_forest__bootstrap' : [True, False],
                    'random_forest__min_samples_leaf': [1,2,5,10]}

forest_grid_search = GridSearchCV(forest_pipeline, 
                                  param_grid=forest_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
forest_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = forest_grid_search.best_score_
best_mae_score = -forest_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][forest_grid_search.best_index_]
best_rmse_score = -forest_grid_search.cv_results_['mean_test_neg_mean_squared_error'][forest_grid_search.best_index_]

# Affichage des résultats
print("Random Forest")
print("-------------------")
print("- Meilleurs hyperparamètres:", forest_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", forest_grid_search.refit_time_  )

#### <font color='skyblue'> Bagging Regressor</font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Bagging
bagging_pipeline = Pipeline([('preprocessor', preprocessor),
                             ('bagging', BaggingRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
bagging_param_grid = {'bagging__n_estimators': [10, 50],  
                     'bagging__max_samples': [0.5, 0.7], 
                     'bagging__max_features': [0.5, 0.7],
                     'bagging__bootstrap': [True, False]}

bagging_grid_search = GridSearchCV(bagging_pipeline, 
                                   param_grid=bagging_param_grid, 
                                   cv=5, 
                                   scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                   refit='r2', 
                                   n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
bagging_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = bagging_grid_search.best_score_
best_mae_score = -bagging_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][bagging_grid_search.best_index_]
best_rmse_score = -bagging_grid_search.cv_results_['mean_test_neg_mean_squared_error'][bagging_grid_search.best_index_]

# Affichage des résultats
print("Bagging Regressor")
print("-------------------")
print("- Meilleurs hyperparamètres:", bagging_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", bagging_grid_search.refit_time_)


- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Bagging
bagging_pipeline = Pipeline([('preprocessor', preprocessor),
                             ('bagging', BaggingRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
bagging_param_grid = {'bagging__n_estimators': [10, 50],  
                     'bagging__max_samples': [0.5, 0.7], 
                     'bagging__max_features': [0.5, 0.7],
                     'bagging__bootstrap': [True, False]}

bagging_grid_search = GridSearchCV(bagging_pipeline, 
                                   param_grid=bagging_param_grid, 
                                   cv=5, 
                                   scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                   refit='r2', 
                                   n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
bagging_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = bagging_grid_search.best_score_
best_mae_score = -bagging_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][bagging_grid_search.best_index_]
best_rmse_score = -bagging_grid_search.cv_results_['mean_test_neg_mean_squared_error'][bagging_grid_search.best_index_]

# Affichage des résultats
print("Bagging Regressor")
print("-------------------")
print("- Meilleurs hyperparamètres:", bagging_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", bagging_grid_search.refit_time_)

#### <font color='skyblue'> Gradient Boost Regressor</font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Gradient Boosting
gb_pipeline = Pipeline([('preprocessor', preprocessor),
                        ('gradient_boosting', GradientBoostingRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
gb_param_grid = {'gradient_boosting__learning_rate': [0.01, 0.1, 0.5], # Taux d'apprentissage
                'gradient_boosting__n_estimators': [50, 100, 200],     # Nombre d'estimateurs (arbres)
                 'gradient_boosting__max_depth': [3, 5, 7],            # Profondeur maximale des arbres
                 'gradient_boosting__min_samples_split': [2, 5, 10],   # Nombre minimal d'échantillons pour la séparation
                 'gradient_boosting__min_samples_leaf': [1, 2, 5]}     # Nombre minimal d'échantillons dans les feuilles

gb_grid_search = GridSearchCV(gb_pipeline, 
                              param_grid=gb_param_grid, 
                              cv=5, 
                              scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                              refit='r2', 
                              n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
gb_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = gb_grid_search.best_score_
best_mae_score = -gb_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][gb_grid_search.best_index_]
best_rmse_score = -gb_grid_search.cv_results_['mean_test_neg_mean_squared_error'][gb_grid_search.best_index_]

# Affichage des résultats
print("Gradient Boosting")
print("-------------------")
print("- Meilleurs hyperparamètres:", gb_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", gb_grid_search.refit_time_  )


- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Gradient Boosting
gb_pipeline = Pipeline([('preprocessor', preprocessor),
                        ('gradient_boosting', GradientBoostingRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
gb_param_grid = {'gradient_boosting__learning_rate': [0.01, 0.1, 0.5], # Taux d'apprentissage
                'gradient_boosting__n_estimators': [50, 100, 200],     # Nombre d'estimateurs (arbres)
                 'gradient_boosting__max_depth': [3, 5, 7],            # Profondeur maximale des arbres
                 'gradient_boosting__min_samples_split': [2, 5, 10],   # Nombre minimal d'échantillons pour la séparation
                 'gradient_boosting__min_samples_leaf': [1, 2, 5]}     # Nombre minimal d'échantillons dans les feuilles

gb_grid_search = GridSearchCV(gb_pipeline, 
                              param_grid=gb_param_grid, 
                              cv=5, 
                              scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                              refit='r2', 
                              n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
gb_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score = gb_grid_search.best_score_
best_mae_score = -gb_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][gb_grid_search.best_index_]
best_rmse_score = -gb_grid_search.cv_results_['mean_test_neg_mean_squared_error'][gb_grid_search.best_index_]

# Affichage des résultats
print("Gradient Boosting logarithme")
print("-------------------")
print("- Meilleurs hyperparamètres:", gb_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score)
print("- Meilleur score MAE :", best_mae_score)
print("- Meilleur score RMSE :", best_rmse_score)
print("- Meilleur temps d'execution:", gb_grid_search.refit_time_  )

#### <font color='skyblue'> Sélection du meilleur modèle </font>

- Sans Normalisation

In [ ]:
# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Liste contenant les modèles et leurs hyperparamètres
measures = []
models_params = [
    (LinearRegression(), {}),
    (DummyRegressor(), {'model__quantile': [0.75], 'model__strategy': ['quantile']}),
    (Lasso(), {'model__alpha': [10.0], 'model__fit_intercept': [True]}),
    (Ridge(), {'model__alpha': [10.0], 'model__fit_intercept': [False]}),
    (RandomForestRegressor(), {'model__bootstrap': [False], 'model__max_depth': [15], 'model__max_features': [0.8], 'model__min_samples_leaf': [1], 'model__min_samples_split': [2]}),
    (BaggingRegressor(), {'model__bootstrap': [False], 'model__max_features': [0.7], 'model__max_samples': [0.5], 'model__n_estimators': [50]}),
    (GradientBoostingRegressor(), {'model__learning_rate': [0.01], 'model__max_depth': [3], 'model__min_samples_leaf': [5], 'model__min_samples_split': [2], 'model__n_estimators': [200]})
]

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Boucle sur chaque modèle
for model, params in models_params:
    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    grid_search = GridSearchCV(pipeline, params, cv=5, 
                              scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                              refit='r2', 
                              n_jobs=-1)
    grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])
    
    # Score R2 MAE RMSE
    best_r2_score = grid_search.best_score_
    best_mae_score = -grid_search.cv_results_['mean_test_neg_mean_absolute_error'][grid_search.best_index_]
    best_rmse_score = -grid_search.cv_results_['mean_test_neg_mean_squared_error'][grid_search.best_index_]

    # Stocker les résultats dans la liste des mesures
    measures.append({'Modèle': model.__class__.__name__,
                     'Meilleur R²': best_r2_score,
                     'Meilleur MAE': best_mae_score,
                     'Meilleur RMSE': best_rmse_score,
                     'Temps d\'exécution': grid_search.refit_time_})

# Création df et affichage
measures_df = pd.DataFrame(measures)
measures_df


- Avec transformation logarithmique

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Liste contenant les modèles et leurs hyperparametres
measures = []
models_params = [(LinearRegression(), {}),
    (DummyRegressor(), {'model__quantile': [0.75], 'model__strategy': ['quantile']}),
    (Lasso(), {'model__alpha': [10.0], 'model__fit_intercept': [True]}),
    (Ridge(), {'model__alpha': [10.0], 'model__fit_intercept': [False]}),
    (RandomForestRegressor(), {'model__bootstrap': [False], 'model__max_depth': [15], 'model__max_features': [0.8], 'model__min_samples_leaf': [1], 'model__min_samples_split': [2]}),
    (BaggingRegressor(), {'model__bootstrap': [False], 'model__max_features': [0.7], 'model__max_samples': [0.5], 'model__n_estimators': [50]}),
    (GradientBoostingRegressor(), {'model__learning_rate': [0.01], 'model__max_depth': [3], 'model__min_samples_leaf': [5], 'model__min_samples_split': [2], 'model__n_estimators': [200]})]

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Boucle sur chaque modèle
for model, params in models_params:
    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    grid_search = GridSearchCV(pipeline, params, cv=5, 
                              scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                              refit='r2', 
                              n_jobs=-1)
    grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])
    
# Score R2 MAE RMSE
    best_r2_score = grid_search.best_score_
    best_mae_score = -grid_search.cv_results_['mean_test_neg_mean_absolute_error'][grid_search.best_index_]
    best_rmse_score = -grid_search.cv_results_['mean_test_neg_mean_squared_error'][grid_search.best_index_]

    measures.append({'Modèle': model.__class__.__name__,
                'Meilleur R²': best_r2_score,
                'Meilleur MAE': best_mae_score,
                'Meilleur RMSE': best_rmse_score,
                'Temps d\'exécution': grid_search.refit_time_})

# création df et affichage
measures_ln_df = pd.DataFrame(measures)
measures_ln_df['Modèle'] = measures_ln_df['Modèle'].apply(lambda x: x + '_ln')
measures_ln_df

In [ ]:
# Concaténation des DataFrames measures_df et measures_ln_df
allmeasures_df = pd.concat([measures_ln_df, measures_df])
allmeasures_df = allmeasures_df.sort_values(by='Modèle')

In [ ]:
allmeasures_df

In [ ]:
# Couleurs par pair
num_pairs = len(allmeasures_df) // 2
color_palette = plt.cm.viridis(np.linspace(0, 1, num_pairs))

# Visualisation
plt.figure(figsize=(20, 16))

# Boucle
for i, metric in enumerate(['Meilleur R²', 'Meilleur MAE', 'Meilleur RMSE', 'Temps d\'exécution']):
    plt.subplot(2, 2, i+1)
    for j in range(0, len(allmeasures_df), 2):
        start_idx = j
        end_idx = min(j + 1, len(allmeasures_df) - 1)
        measures_data_plot = allmeasures_df.iloc[start_idx:end_idx+1]
        color_idx = j // 2
        color = color_palette[color_idx]
        plt.bar(measures_data_plot['Modèle'], measures_data_plot[metric], color=color, label=f'Pair {color_idx + 1}')
    plt.title(f'{metric} par Modèle')
    plt.xlabel('Modèle')
    plt.ylabel(metric)
    plt.xticks(rotation=90)
    plt.legend()

plt.tight_layout()
plt.show()



#### <font color='skyblue'> Choix du Random Forest Regressor </font>

- Feature importance

In [ ]:
# Exécution de la recherche sur la grille des hyperparamètres
forest_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Récupérez le meilleur modèle entraîné
best_model = forest_grid_search.best_estimator_

# Récupérez les noms des caractéristiques numériques à partir du préprocesseur
numerical_feature_names = best_model.named_steps['preprocessor'].transformers_[0][2]

# Récupérez les noms des caractéristiques catégorielles encodées
categorical_feature_names = best_model.named_steps['preprocessor'].named_transformers_['categorical'].named_steps['onehot'].get_feature_names_out()

# Concaténez les noms des caractéristiques catégorielles et numériques
feature_names = np.concatenate([numerical_feature_names, categorical_feature_names])

# Récupérez l'importance des caractéristiques à partir du meilleur modèle
feature_importance = best_model.named_steps['random_forest'].feature_importances_

# Créez un DataFrame pour afficher l'importance des caractéristiques
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Triez les caractéristiques par ordre d'importance décroissante
importance_df = importance_df.sort_values(by='Importance', ascending=False)


# Visualisez les 10 caractéristiques les plus importantes
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Caractéristique')
plt.title('Top 10 des Caractéristiques les Plus Importantes')
plt.gca().invert_yaxis()  # Inverser l'axe y pour avoir les caractéristiques les plus importantes en haut
plt.show()


In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Random Forest
forest_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('random_forest', RandomForestRegressor())])

# Calcul des scores à l'aide de la validation croisée
cv_scores_r2 = cross_val_score(forest_pipeline, X_train, y_train['SiteEnergyUse(kBtu)'], cv=5, scoring='r2')
cv_scores_mae = cross_val_score(forest_pipeline, X_train, y_train['SiteEnergyUse(kBtu)'], cv=5, scoring='neg_mean_absolute_error')
cv_scores_rmse = cross_val_score(forest_pipeline, X_train, y_train['SiteEnergyUse(kBtu)'], cv=5, scoring='neg_mean_squared_error')

# Affichage des scores moyens
print("R2 moyen:", np.mean(cv_scores_r2))
print("MAE moyen:", -np.mean(cv_scores_mae))
print("RMSE moyen:", np.sqrt(-np.mean(cv_scores_rmse)))

# Visualisation des scores
plt.figure(figsize=(10, 6))

# Score R2
plt.subplot(2, 2, 1)
plt.plot(cv_scores_r2, marker='o')
plt.title('Scores R2')
plt.xlabel('Fold')
plt.ylabel('Score')

# Score MAE
plt.subplot(2, 2, 2)
plt.plot(-cv_scores_mae, marker='o')
plt.title('Scores MAE')
plt.xlabel('Fold')
plt.ylabel('Score')

# Score RMSE
plt.subplot(2, 2, 3)
plt.plot(np.sqrt(-cv_scores_rmse), marker='o')
plt.title('Scores RMSE')
plt.xlabel('Fold')
plt.ylabel('Score')

plt.tight_layout()
plt.show()

In [ ]:
best_rf_model = forest_grid_search.best_estimator_
best_rf_model.fit(X_train, y_train['SiteEnergyUse(kBtu)'])
test_predictions = best_rf_model.predict(X_test)

# Plotting the Model Predictios vs True Values
plt.figure(figsize=(10, 6))
plt.scatter(y_test['SiteEnergyUse(kBtu)'], y_test['SiteEnergyUse(kBtu)'], alpha=0.5, color='blue', label='True Values')
plt.scatter(y_test['SiteEnergyUse(kBtu)'], test_predictions, alpha=0.5, color='red', label='Model Predictions')
plt.title('Valeurs réélles vs prédictions du modèle')
plt.xlabel('Valeurs réélles')
plt.ylabel('Prédictions')
plt.legend()
plt.show()

#### <font color='skyblue'> Importance de la variable EnergyStarScore </font>

In [ ]:
columns_to_drop = ['OSEBuildingID', 'PropertyName', 'Address','Unnamed: 0','CouncilDistrictCode',
                   'TaxParcelIdentificationNumber','YearBuilt','ListOfAllPropertyUseTypes',
                   'SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType', 'PrimaryPropertyType',
                   'CouncilDistrictCode','YearBuilt','PropertyGFABuilding(s)',
                   'LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseTypeGFA',
                    'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)','SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 
                    'Electricity(kWh)', 'NaturalGas(therms)',
                    'GHGEmissionsIntensity','SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)','LargestPropertyUseType',
                    'BuildingType','PropertyGFAParking']

data_bis.drop(columns=columns_to_drop, inplace=True)

# Supprimer les lignes contenant des valeurs NaN dans la colonne 'ENERGYSTARScore'
data_bis.dropna(subset=['ENERGYSTARScore'], inplace=True)

categorical_data = ['Category', 'Neighborhood']
numerical_data = ['NumberofFloors','NumberofBuildings','PropertyGFATotal','BuildingAge','GasUseRatio','ElectricityUseRatio','ParkingGFARatio']

X = data_bis.drop(['TotalGHGEmissions','SiteEnergyUse(kBtu)'], axis=1)

y = data_bis[['TotalGHGEmissions','SiteEnergyUse(kBtu)']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

print('X_train :', X_train.shape)
print('y_train :', y_train.shape)
print('X_test :' , X_test.shape)
print('y_test :',y_test.shape)

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Random Forest
forest_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('random_forest', RandomForestRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
forest_param_grid = {'random_forest__bootstrap': [False], 'random_forest__max_depth': [15], 'random_forest__max_features': [0.8], 'random_forest__min_samples_leaf': [1], 'random_forest__min_samples_split': [2]}

forest_grid_search = GridSearchCV(forest_pipeline, 
                                  param_grid=forest_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
forest_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score1 = forest_grid_search.best_score_
best_mae_score1 = -forest_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][forest_grid_search.best_index_]
best_rmse_score1 = -forest_grid_search.cv_results_['mean_test_neg_mean_squared_error'][forest_grid_search.best_index_]

# Affichage des résultats
print("Random Forest")
print("-------------------")
print("- Meilleurs hyperparamètres:", forest_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score1)
print("- Meilleur score MAE :", best_mae_score1)
print("- Meilleur score RMSE :", best_rmse_score1)
print("- Meilleur temps d'execution:", forest_grid_search.refit_time_)

Comparaison avec données de test

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Création du pipeline du modèle Random Forest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('random_forest', RandomForestRegressor())])

# Entraînement du modèle
pipeline.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Prédiction sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculez le coefficient de détermination R²
r2 = r2_score(y_test['SiteEnergyUse(kBtu)'], y_pred)

print("Coefficient de détermination R² sur les données de test :", r2)

In [ ]:
categorical_data = ['Category', 'Neighborhood']
numerical_data = ['NumberofFloors','NumberofBuildings','PropertyGFATotal','BuildingAge','GasUseRatio','ElectricityUseRatio','ParkingGFARatio','ENERGYSTARScore']

X = data_bis.drop(['TotalGHGEmissions','SiteEnergyUse(kBtu)'], axis=1)

y = data_bis[['TotalGHGEmissions','SiteEnergyUse(kBtu)']]

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Pipeline de Random Forest
forest_pipeline = Pipeline([('preprocessor', preprocessor),
                            ('random_forest', RandomForestRegressor())])

# Création des métriques de score personnalisées pour R2 MAE RMSE
scorer_r2 = make_scorer(r2_score)
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
scorer_rmse = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)

# Hyperparamètres à rechercher
forest_param_grid = {'random_forest__bootstrap': [False], 'random_forest__max_depth': [15], 'random_forest__max_features': [0.8], 'random_forest__min_samples_leaf': [1], 'random_forest__min_samples_split': [2]}

forest_grid_search = GridSearchCV(forest_pipeline, 
                                  param_grid=forest_param_grid, 
                                  cv=5, 
                                  scoring={'r2': scorer_r2, 'neg_mean_absolute_error': scorer_mae, 'neg_mean_squared_error': scorer_rmse}, 
                                  refit='r2', 
                                  n_jobs=-1)

# Exécution de la recherche sur la grille des hyperparamètres
forest_grid_search.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Score R2 MAE RMSE
best_r2_score2 = forest_grid_search.best_score_
best_mae_score2 = -forest_grid_search.cv_results_['mean_test_neg_mean_absolute_error'][forest_grid_search.best_index_]
best_rmse_score2 = -forest_grid_search.cv_results_['mean_test_neg_mean_squared_error'][forest_grid_search.best_index_]

# Affichage des résultats
print("Random Forest")
print("-------------------")
print("- Meilleurs hyperparamètres:", forest_grid_search.best_params_)
print("- Meilleur score R2 :", best_r2_score2)
print("- Meilleur score MAE :", best_mae_score2)
print("- Meilleur score RMSE :", best_rmse_score2)
print("- Meilleur temps d'execution:", forest_grid_search.refit_time_)

Comparaison avec données de test

In [ ]:
# Définir la fonction de transformation logarithmique
def logarithmic_transform(X): return np.log1p(X)

# Pipeline de transformations
numerical_pipeline = Pipeline(steps=[('log_transform', FunctionTransformer(func=logarithmic_transform))])
categorical_pipeline = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('numerical', numerical_pipeline, numerical_data),
                                               ('categorical', categorical_pipeline, categorical_data)])

# Création du pipeline du modèle Random Forest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('random_forest', RandomForestRegressor())])

# Entraînement du modèle
pipeline.fit(X_train, y_train['SiteEnergyUse(kBtu)'])

# Prédiction sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Calculez le coefficient de détermination R²
r2 = r2_score(y_test['SiteEnergyUse(kBtu)'], y_pred)

print("Coefficient de détermination R² sur les données de test :", r2)

In [ ]:
# Données à comparer
measures_with_energystarscore = {'R2': best_r2_score1,
                                    'MAE': best_mae_score1,
                                    'RMSE': best_rmse_score1,
                                    'Temps d\'exécution': forest_grid_search.refit_time_}

measures_without_energystarscore  = {'R2': best_r2_score2,
                                'MAE': best_mae_score2,
                                'RMSE': best_rmse_score2,
                                'Temps d\'exécution': forest_grid_search.refit_time_}

# Création de la figure et des sous-graphiques
plt.figure(figsize=(8, 6))

plt.subplot(2, 2, 1)
plt.bar(['Sans Energystarscore', 'Avec Energystarscore'], [measures_without_energystarscore['R2'], measures_with_energystarscore['R2']], color=['steelblue', 'darkblue'])
plt.title('Comparaison de R2')
plt.ylabel('R2')

plt.subplot(2, 2, 2)
plt.bar(['Sans Energystarscore', 'Avec Energystarscore'], [measures_without_energystarscore['MAE'], measures_with_energystarscore['MAE']], color=['steelblue', 'darkblue'])
plt.title('Comparaison de MAE')
plt.ylabel('MAE')

plt.subplot(2, 2, 3)
plt.bar(['Sans Energystarscore', 'Avec Energystarscore'], [measures_without_energystarscore['RMSE'], measures_with_energystarscore['RMSE']], color=['steelblue', 'darkblue'])
plt.title('Comparaison de RMSE')
plt.ylabel('RMSE')

plt.subplot(2, 2, 4)
plt.bar(['Sans Energystarscore', 'Avec Energystarscore'], [measures_without_energystarscore['Temps d\'exécution'], measures_with_energystarscore['Temps d\'exécution']], color=['steelblue', 'darkblue'])
plt.title('Comparaison de temps d\'exécution')
plt.ylabel('Temps d\'exécution')
plt.tight_layout()
plt.show()